In [18]:
import pandas as pd
from sqlalchemy import create_engine
from mysql.connector import Error

In [58]:
#DB Connection 

DB_NAME = 'ProjectX'
userName = 'project2'
userPass = 'Project2'
DBport = '3306'

#!pip install pymysql
# use pymysql as connector

connectionString = "mysql+pymysql://{}:{}@localhost:{}/".format(userName, userPass, DBport)

connectionDB = None

try:
    connection = create_engine(connectionString)
    connectionDB = connection.connect()
    connectionDB.execute("commit")
    record = connection.execute("CREATE DATABASE IF NOT EXISTS {};".format(DB_NAME))
    print("Connected to MySQL and created DB {} \n{}".format(DB_NAME, connectionDB))
    
except Error as e :
    print ("Error while connecting to MySQL", e)  

Connected to MySQL and created DB ProjectX 


In [59]:
#Table layout

TABLES = {}

tableSyntax = "CREATE TABLE IF NOT EXISTS"
TABLES['CarSales'] = (
    "{} `CarSales` ("
    " `id` int(6) NOT NULL AUTO_INCREMENT,"
    " `Brand` varchar(10) NOT NULL,"
    " `Model` varchar(10) NOT NULL,"
    " `VehicleType` varchar(15) NOT NULL,"
    " `SaleVolume` int(10) NOT NULL,"
    " `Date` varchar(10) NOT NULL,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB".format(tableSyntax) )

TABLES['TotalSales'] = (
    "{} `TotalSales` ("
    "  `id` int(6) NOT NULL AUTO_INCREMENT,"
    "  `Date`  DATE NOT NULL,"
    "  `Volume` INT(10) NOT NULL,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB".format(tableSyntax) )

In [60]:
# create the tables in mySQL

try:
    connectionDB.execute('Use {};'.format(DB_NAME))
    for eachTable in TABLES.keys():
        table_description = TABLES[eachTable]
        #print(eachTable, table_description)
        try:
            print("Creating table {}: ".format(table_description), end='')
            connectionDB.execute(table_description)
        except Error as err:
            print("Failed creating table: {}".format(err))
except:
    print("Connection is not working")
        
#connectionDB.close()
#close the connection when done

Creating table CREATE TABLE IF NOT EXISTS `CarSales` ( `id` int(6) NOT NULL AUTO_INCREMENT, `Brand` varchar(10) NOT NULL, `Model` varchar(10) NOT NULL, `VehicleType` varchar(15) NOT NULL, `SaleVolume` int(10) NOT NULL, `Date` varchar(10) NOT NULL,  PRIMARY KEY (`id`)) ENGINE=InnoDB: Creating table CREATE TABLE IF NOT EXISTS `TotalSales` (  `id` int(6) NOT NULL AUTO_INCREMENT,  `Date`  DATE NOT NULL,  `Volume` INT(10) NOT NULL,  PRIMARY KEY (`id`)) ENGINE=InnoDB: 

In [61]:
df = pd.read_csv('TOTALSA.csv')
df.columns = ['Date', 'SaleVol']
df['SaleVol'] = df['SaleVol'] * 1000
df['SaleVol']= df['SaleVol'].astype(int)
df.dtypes
#df

Date       object
SaleVol     int32
dtype: object

In [62]:
connectionDBString = connectionString+DB_NAME
connectionDB = create_engine(connectionDBString)

df.to_sql(con=connectionDB, name='totalsales', if_exists='replace')
